# SQL query to clickhouse database

**Задание:**  
В базе данных вашей компании присутствуют следующие таблицы  
- city
    - city_id – id города, где проходит акция
    - client_city_id – id города, где находится покупатель
    - city – город
    - client_city – город, где находится покупатель
- client
    - client_id – id покупателя
    - client_city_id – id города, где находится покупатель
    - birth_date – дата рождения покупателя
    - registration – дата регистрации покупателя
- promotion
    - promotion_id – id акции
    - category_id – id категории
    - promotion_name – акция
    - category_name – категория
    - partner_id – id партнёра
    - partner_name – партнёр
- purchase
    - purchase_id – id покупки
    - partner_id – id партнёра
    - client_id – id покупателя
    - city_id – id города
    - promotion_id – id акции
    - category_id – id категории
    - purchase_date – дата покупки
    - price – цена за единицу товара
    - quantity – число проданных единиц
    - status – статус покупки  

Написать запрос, чтобы получить такую таблицу:  
- purchase_date – дата покупки
- purchase_id – id покупки
- client_id – id покупателя
- client_age – возраст покупателя
- client_registration_age – как долго человек пользуется вашими магазинами
- client_category – содержит new или old, если это первая или последующая покупка соответственно
- promotion_name – акция
- category_name – категория
- partner_name – партнёр
- client_city – город, где находится покупатель
- city – город
- revenue – сумма выручки
- quantity – число проданных единиц  

При этом в таблице должны присутствовать только значения, где в поле status стоит значение 1, и только те, где purchase_date находится в диапазоне от 01.05.2020 до 01.08.2020.  
База данных доступна по имени project_variant_2

In [1]:
import pandas as pd
import pandahouse as ph

Задаём параметры подключения к базе данных

In [2]:
connection = dict(database='default',
                  host='https://clickhouse.lab.karpov.courses',
                  user='student',
                  password='dpo_python_2020')

Создаём датафрейм итоговой таблицы по SQL запросу

In [3]:
query = """
SELECT
    purch.purchase_date AS purchase_date,
    purch.purchase_id AS purchase_id,
    purch.client_id AS client_id,
    dateDiff(YEAR, cl.birth_date, now()) - IF(DayOfYear(cl.birth_date) <= DayOfYear(now()), 0, 1)
        AS client_age,
    dateDiff(YEAR, cl.registration, now()) - IF(DayOfYear(cl.registration) <= DayOfYear(now()), 0, 1)
        AS client_registration_age,
    if(purch.purchase_id = cl_cat.min_purchase_id, 'new', 'old') AS client_category,
    prom.promotion_name AS promotion_name,
    prom.category_name AS category_name,
    prom.partner_name AS partner_name,
    cl_city.client_city AS client_city,
    prom_city.city AS city,
    CAST(purch.price AS Float32) * CAST(purch.quantity AS Float32) AS revenue,
    CAST(purch.quantity AS Float32) AS quantity
FROM project_variant_2.purchase AS purch

JOIN project_variant_2.client AS cl
        ON purch.client_id = cl.client_id
 
JOIN
    (
    SELECT
        MIN(l.purchase_id) AS min_purchase_id,
        l.client_id AS client_id
    FROM project_variant_2.purchase AS l
    JOIN
        (
        SELECT 
            MIN(purchase_date) AS min_date,
            client_id
        FROM project_variant_2.purchase
        GROUP BY client_id
        ) AS r
            ON l.client_id = r.client_id
    WHERE l.purchase_date = r.min_date
    GROUP BY l.client_id
    ) AS cl_cat
        ON purch.client_id = cl_cat.client_id       

LEFT JOIN project_variant_2.promotion AS prom
        ON      purch.promotion_id = prom.promotion_id
            AND purch.category_id = prom.category_id
            AND purch.partner_id = prom.partner_id

JOIN
    (
    SELECT *
    FROM project_variant_2.client AS l
    ANY LEFT JOIN project_variant_2.city AS r
        ON l.client_city_id = r.client_city_id
    ) AS cl_city
        ON purch.client_id = cl_city.client_id

LEFT ANY JOIN 
    (
    SELECT *
    FROM project_variant_2.city) AS prom_city
        ON purch.city_id = prom_city.city_id

WHERE 
        purch.status = 1
    AND purch.purchase_date >= '2020-05-01'
    AND purch.purchase_date < '2020-08-01'
    
ORDER BY purchase_date
"""
df = ph.read_clickhouse(query, connection=connection)

In [4]:
df.shape

(2695, 13)

In [5]:
df.head()

,purchase_date,purchase_id,client_id,client_age,client_registration_age,client_category,promotion_name,category_name,partner_name,client_city,city,revenue,quantity
0,2020-05-01,470,918,23,1,old,,,,Щучинщина,Ярославль,660.0,4.0
1,2020-05-01,1210,126,29,2,old,campaign_3,Овощи,Всем партнёрам партнёр,Щучинщина,Ярославль,225.0,1.0
2,2020-05-01,1675,694,20,1,old,,,,Вена,Казань,520.0,8.0
3,2020-05-01,1703,630,30,2,old,,,,Ярославль,Москва,55.0,1.0
4,2020-05-01,1761,841,24,1,old,,,,Ярославль,Минск,225.0,1.0


In [6]:
df.dtypes

purchase_date              datetime64[ns]
purchase_id                         int32
client_id                           int32
client_age                          int64
client_registration_age             int64
client_category                    object
promotion_name                     object
category_name                      object
partner_name                       object
client_city                        object
city                               object
revenue                           float64
quantity                          float32
dtype: object

Записываем итоговую таблицу в файл csv

In [7]:
df.to_csv('final_table.csv', index=False)